In [1]:
import os
import pandas as pd
import numpy as np
import audiolabel

ImportError: No module named 'PyQt4.QtGui'

In [208]:
datadir = '../resources/'

fnames = [f.replace('.tg', '') for f in os.listdir(datadir) if f.endswith('.tg')]
fnames

['two_plus_two', 'three_plus_five']

In [209]:
fname = fnames[0]

tg = os.path.join(datadir, fname + '.tg')
phdf, wddf = audiolabel.LabelManager(tg, from_type='praat').as_df()

In [186]:
phdf = phdf.assign(fname=fname)
phdf

,t1,t2,text,duration,center,fname
0,0.0125,0.3417,T,0.3292,0.17710,two_plus_two
1,0.3417,0.4914,UW1,0.1497,0.41655,two_plus_two
2,0.4914,0.5912,P,0.0998,0.54130,two_plus_two
3,0.5912,0.6211,L,0.0299,0.60615,two_plus_two
4,0.6211,0.6909,AH1,0.0698,0.65600,two_plus_two
5,0.6909,0.8805,S,0.1896,0.78570,two_plus_two
6,0.8805,0.9803,T,0.0998,0.93040,two_plus_two
7,0.9803,1.3195,UW1,0.3392,1.14990,two_plus_two
8,1.3195,1.3594,sp,0.0399,1.33945,two_plus_two
9,1.3594,1.4193,IY1,0.0599,1.38935,two_plus_two


In [187]:
# Extract phone and stress from text and concat to phdf.
phdf = pd.concat(
    [
        phdf,
        phdf.text.str.extract(r'(?P<phone>[^\d]+)(?P<stress>\d*)', expand=True)
    ],
    axis='index'
)
phdf

,t1,t2,text,duration,center,fname,phone,stress
0,0.0125,0.3417,T,0.3292,0.17710,two_plus_two,T,
1,0.3417,0.4914,UW1,0.1497,0.41655,two_plus_two,UW,1
2,0.4914,0.5912,P,0.0998,0.54130,two_plus_two,P,
3,0.5912,0.6211,L,0.0299,0.60615,two_plus_two,L,
4,0.6211,0.6909,AH1,0.0698,0.65600,two_plus_two,AH,1
5,0.6909,0.8805,S,0.1896,0.78570,two_plus_two,S,
6,0.8805,0.9803,T,0.0998,0.93040,two_plus_two,T,
7,0.9803,1.3195,UW1,0.3392,1.14990,two_plus_two,UW,1
8,1.3195,1.3594,sp,0.0399,1.33945,two_plus_two,sp,
9,1.3594,1.4193,IY1,0.0599,1.38935,two_plus_two,IY,1


0     False
1      True
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17     True
18    False
Name: phone, dtype: bool

In [207]:
# Add preceding, following phone context columns.
phdf = phdf.assign(post=phdf.phone, pre=phdf.phone)
phdf.post = phdf.post.shift(-1)
phdf.pre = phdf.pre.shift(1)
phdf[(phdf.phone.isin(['UW', 'AO'])) & (phdf.pre.isin(['T', 'F']))]

,t1,t2,text,duration,center,fname,phone,stress,post,pre
1,0.3417,0.4914,UW1,0.1497,0.41655,two_plus_two,UW,1,P,T
7,0.9803,1.3195,UW1,0.3392,1.14990,two_plus_two,UW,1,sp,T
17,1.8683,2.0678,AO1,0.1995,1.96805,two_plus_two,AO,1,R,F


In [152]:
ipadf = pd.read_csv(
    os.path.join(datadir, 'arpabet2ipa.txt'),
    sep='\t',
    names=('phone', 'ipa')
)
ipadf

,phone,ipa
0,AO,ɔ
1,AO0,ɔ
2,AO1,ɔ
3,AO2,ɔ
4,AA,ɑ
5,AA0,ɑ
6,AA1,ɑ
7,AA2,ɑ
8,IY,i
9,IY0,i


In [153]:
phdf = phdf.merge(ipadf, on='phone', how='left')
phdf

,t1,t2,text,duration,center,fname,phone,stress,ipa
0,0.0125,0.3417,T,0.3292,0.17710,two_plus_two,T,,t
1,0.3417,0.4914,UW1,0.1497,0.41655,two_plus_two,UW,1,u
2,0.4914,0.5912,P,0.0998,0.54130,two_plus_two,P,,p
3,0.5912,0.6211,L,0.0299,0.60615,two_plus_two,L,,l
4,0.6211,0.6909,AH1,0.0698,0.65600,two_plus_two,AH,1,ʌ
5,0.6909,0.8805,S,0.1896,0.78570,two_plus_two,S,,s
6,0.8805,0.9803,T,0.0998,0.93040,two_plus_two,T,,t
7,0.9803,1.3195,UW1,0.3392,1.14990,two_plus_two,UW,1,u
8,1.3195,1.3594,sp,0.0399,1.33945,two_plus_two,sp,,NaN
9,1.3594,1.4193,IY1,0.0599,1.38935,two_plus_two,IY,1,i


In [123]:
phgrp = phdf.groupby('ipa')

In [124]:
phgrp.size()

ipa
f    1
i    1
k    1
l    2
p    1
r    1
s    1
t    2
u    2
w    1
z    1
ɔ    1
ʌ    2
dtype: int64

In [80]:
# Store count of rows in each group. count() returns number of non-missing rows per group,
# so take care to choose a column that cannot contain nan.
ct = phgrp.size()
ct.index

Index(['AH', 'AO', 'F', 'IY', 'K', 'L', 'P', 'R', 'S', 'T', 'UW', 'W', 'Z',
       'sp'],
      dtype='object', name='phone')

In [108]:
summarydf = phgrp.mean()
summarydf = summarydf.assign(n=ct)
summarydf #.merge(phgrp.max())
phgrp.max()

,t1,t2,text,duration,center,fname,stress
phone,,,,,,,
AH,1.5789,1.6088,AH1,0.0698,1.59385,two_plus_two,1
AO,1.8683,2.0678,AO1,0.1995,1.96805,two_plus_two,1
F,1.8283,1.8683,F,0.0400,1.84830,two_plus_two,
IY,1.3594,1.4193,IY1,0.0599,1.38935,two_plus_two,1
K,1.4193,1.5290,K,0.1097,1.47415,two_plus_two,
L,1.6088,1.6488,L,0.0400,1.62880,two_plus_two,
P,0.4914,0.5912,P,0.0998,0.54130,two_plus_two,
R,2.0678,2.1975,R,0.1297,2.13265,two_plus_two,
S,0.6909,0.8805,S,0.1896,0.78570,two_plus_two,
